In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/ubuntu/open-domain-dialog


In [2]:
from odd.gpt.gpt import GPTTask

/home/ubuntu/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
task = GPTTask.load_from_checkpoint(
    "checkpoint/open-domain-dialog/kodialogpt-base-v1/last.ckpt",
    )
task

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_271016/1928061496.py:1 in <module>                                         │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_271016/1928061496.py'                │
│ /home/ubuntu/.local/lib/python3.9/site-packages/pytorch_lightning/core/saving.py:139 in   │
│ load_from_checkpoint                                                                      │
│                                                                                           │
│   136 │   │   │   if map_location is not None:                                            │
│   137 │   │   │   │   checkpoint = pl_load(checkpoint_path, map_location=map_location)    │
│   138 │   │   │   else:                                                                   │
│ ❱ 139 │   │   │   │   checkpoint = pl_load(checkpoint_path, map_location=lambda storage,  │
│   140 │   │                                                                               │
│   141 │   │   if hparams_file is not None:                                                │
│   142 │   │   │   extension = hparams_file.split(".")[-1]                                 │
│                                                                                           │
│ /home/ubuntu/.local/lib/python3.9/site-packages/pytorch_lightning/utilities/cloud_io.py:4 │
│ 7 in load                                                                                 │
│                                                                                           │
│   44 │   │   return torch.hub.load_state_dict_from_url(str(path_or_url), map_location=map │
│   45 │   fs = get_filesystem(path_or_url)                                                 │
│   46 │   with fs.open(path_or_url, "rb") as f:                                            │
│ ❱ 47 │   │   return torch.load(f, map_location=map_location)                              │
│   48                                                                                      │
│   49                                                                                      │
│   50 def get_filesystem(path: _PATH, **kwargs: Any) -> AbstractFileSystem:                │
│                                                                                           │
│ /home/ubuntu/.local/lib/python3.9/site-packages/torch/serialization.py:712 in load        │
│                                                                                           │
│    709 │   │   │   │   │   │   │   │     " silence this warning)", UserWarning)           │
│    710 │   │   │   │   │   opened_file.seek(orig_position)                                │
│    711 │   │   │   │   │   return torch.jit.load(opened_file)                             │
│ ❱  712 │   │   │   │   return _load(opened_zipfile, map_location, pickle_module, **pickle │
│    713 │   │   return _legacy_load(opened_file, map_location, pickle_module, **pickle_loa │
│    714                                                                                    │
│    715                                                                                    │
│                                                                                           │
│ /home/ubuntu/.local/lib/python3.9/site-packages/torch/serialization.py:1046 in _load      │
│                                                                                           │
│   1043 │                                                                                  │
│   1044 │   unpickler = UnpicklerWrapper(data_file, **pickle_load_args)                    │
│   1045 │   unpickler.persistent_load = persistent_load                                    │
│ ❱ 1046 │   result = unpickler.load()              

In [16]:
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    min_new_tokens=16,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

print(task.generate(["0 : 안녕하세요 \n 1 : 맙소사. 오늘 날씨 좋네요. 괜찮으세요? \n"], **generation_args)[0])

0 : 네 오늘은 날씨가 좋았어요. 구름도 예뻤습니다. 



In [8]:
task.model.save_pretrained("huggingface/kodialogpt-v0/")
task.tokenizer.save_pretrained("huggingface/kodialogpt-v0/")

('huggingface/kodialogpt-v0/tokenizer_config.json',
 'huggingface/kodialogpt-v0/special_tokens_map.json',
 'huggingface/kodialogpt-v0/vocab.json',
 'huggingface/kodialogpt-v0/merges.txt',
 'huggingface/kodialogpt-v0/added_tokens.json',
 'huggingface/kodialogpt-v0/tokenizer.json')

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [9]:
model = "huggingface/kodialogpt-v0/"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

[[{'generated_text': '0 : **는 게임 좋아하니\n1 : 나는 게임을 잘 안 해 키키 '}],
 [{'generated_text': '0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 : 아이고... 진짜 무섭다... '}],
 [{'generated_text': '0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : 뭘 잘못 했길래 그래? '}]]

In [12]:
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    min_new_tokens=32,
    do_sample=True,
    top_k=50,
    early_stopping=True
)
generator(
    ["0 : **는 게임 좋아하니\n1 :",
    "0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 :",
    "0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : "],
    **generation_args
)

[[{'generated_text': '0 : **는 게임 좋아하니\n1 : 난 게임을 잘 안 해 키키 '}],
 [{'generated_text': '0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 : 아 진짜? 근데 그건 좀 아니지 않아? '}],
 [{'generated_text': '0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : 뭘 잘못 했길래 그래? '}]]